In [1]:
!pip install uproot awkward 
!pip install xgboost
from uproot_io import Events, View
import numpy as np
import matplotlib as plt
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from scipy.spatial import ConvexHull
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report
import xgboost as xgb
import joblib

In [ ]:
"""
BDT Feature Functions
""" 
# If a pfo doesnt have 15 hits, then NONE is returned for ALL FEATURES

# track/shower
def correlation(events, event_idx, min_hits=15):
    x_hits = events.reco_hits_x_w[event_idx]
    w_hits = events.reco_hits_w[event_idx] 

    # Check if there are valid hits
    if len(w_hits) == len(x_hits) and len(w_hits) > min_hits: # talk about advantages and disadvantages of results with a threshold 
        if np.std(x_hits) == 0 or np.std(w_hits) == 0:
            return None  # No valid correlation if there's no variation in data
        
        correlation = np.corrcoef(x_hits, w_hits)[0, 1]
        
        # Fit line using w_hits for x and calculate predicted y-values
        line_fit = np.polyfit(w_hits, x_hits, 1)
        line_y_pred = np.polyval(line_fit, w_hits)
        
        # Calculate line error between predicted and actual x_hits
        line_error = np.mean((x_hits - line_y_pred) ** 2)
        
        # Normalise scores
        correlation_score = abs(correlation) if not np.isnan(correlation) else 0
        error_score = max(0, 1 - line_error / 20) if line_error < 20 else 0
        
        # Weighted score
        line_score = (correlation_score * 0.7) + (error_score * 0.3)
        
        return (line_score * 100)  # Return the score and category

    else:
        return None

def noise(events, event_idx, min_hits=15, eps=2, min_samples=5):
    # Extract hit positions (no pdg filtering, just use reco hits)
    x_hits = events.reco_hits_x_w[event_idx]
    w_hits = events.reco_hits_w[event_idx]

    # Check if there are valid hits
    if len(w_hits) == len(x_hits) and len(w_hits) > min_hits:
        # Combine the coordinates for clustering
        hits_coordinates = np.column_stack((w_hits, x_hits))

        # Apply DBSCAN clustering
        db = DBSCAN(eps=eps, min_samples=min_samples).fit(hits_coordinates)
        labels = db.labels_

        # Count noise points (labeled as -1)
        n_noise = np.sum(labels == -1)

        # Count clusters (unique labels excluding -1)
        unique_clusters = set(labels) - {-1}
        n_clusters = len(unique_clusters)

        return n_noise + n_clusters
    else:
        return None
        
def rms(events, event_idx, min_hits=15):
    w_hits = events.reco_hits_w[event_idx]
    x_hits = events.reco_hits_x_w[event_idx]

    if len(w_hits) == len(x_hits) and len(w_hits) > min_hits:
        slope, intercept = np.polyfit(w_hits, x_hits, 1)
        
        actual = x_hits
        predicted = slope * w_hits + intercept
        
        meanSquaredError = ((predicted - actual) ** 2).mean()
        return np.sqrt(meanSquaredError)
    else: 
        return None 
        
def angle(events, event_idx, min_hits=15):
    x_hits = events.reco_hits_x_w[event_idx]
    w_hits = events.reco_hits_w[event_idx]

    if len(w_hits) == len(x_hits) and len(w_hits) > min_hits:
        # Fit the best-fit line
        line_fit = np.polyfit(w_hits, x_hits, 1)
        line_slope = line_fit[0]
        line_intercept = line_fit[1]

        # Calculate residuals (distance from the line)
        line_y_pred = np.polyval(line_fit, w_hits)
        residuals = np.abs(x_hits - line_y_pred)

        # Find the index of the furthest point
        furthest_idx = np.argmax(residuals)
        furthest_point = np.array([x_hits[furthest_idx], w_hits[furthest_idx]])

        # Start of the line is at the minimum W-coordinate
        min_w = np.min(w_hits)
        start_point = np.array([line_slope * min_w + line_intercept, min_w])

        # End of the red line (best-fit line) at the maximum W-coordinate
        max_w = np.max(w_hits)
        end_of_red_line = np.array([line_slope * max_w + line_intercept, max_w])

        # Calculate the lengths of the three sides of the triangle
        red_line_length = np.linalg.norm(end_of_red_line - start_point)  # Distance between start and end of red line
        purple_line_length = np.linalg.norm(furthest_point - start_point)  # Distance between start and furthest point (purple line)
        third_line_length = np.linalg.norm(furthest_point - end_of_red_line)  # Distance between end of red line and furthest point (third line)

        # Using the cosine rule to calculate the angle between the red and purple lines
        cos_theta = (red_line_length**2 + purple_line_length**2 - third_line_length**2) / (2 * red_line_length * purple_line_length)
        angle_radians = np.arccos(np.clip(cos_theta, -1.0, 1.0))  # Clip value to avoid out-of-bound errors
        angle_degrees = np.degrees(angle_radians)  # Convert radians to degrees
        
        return angle_degrees
    else:
        return None
        
def line(events, event_idx, min_hits=15):
    w_hits = np.array(events.reco_hits_w[event_idx])
    x_hits = np.array(events.reco_hits_x_w[event_idx])

    if len(w_hits) == len(x_hits) and len(w_hits) > 15:
    
        # Calculate differences between consecutive points
        dx = np.diff(w_hits)
        dy = np.diff(x_hits)
    
        # Compute segment lengths
        segment_lengths = np.sqrt(dx**2 + dy**2)
    
        # Total arc length (line integral)
        total_length = np.sum(segment_lengths)
    
        # Normalize by the number of points
        normalised_length = total_length / len(w_hits)

        return normalised_length
    else:
        return None 
        
def q4(events, event_idx, min_hits=15):
    adcs = events.reco_adcs_w[event_idx]

    if len(adcs) > 15:

        q4_idx = len(adcs) // 4

        adcs_q4 = adcs[-q4_idx:]
    
        ratio = sum(adcs_q4) / sum(adcs)
    
    
        return ratio
    else:
       return None

# e/gamma
def step_length(events, event_idx, min_hits=15):
    # Find all info for the feature
    w_hits = events.reco_hits_w[event_idx]
    x_hits = events.reco_hits_x_w[event_idx]
    w_vtx = events.neutrino_vtx_w[event_idx]
    x_vtx = events.neutrino_vtx_x[event_idx]

    # Skip events where there are fewer than 15 hits
    if len(w_hits) <= min_hits:
        return None  # Return None to indicate that the feature should be skipped

    # Finding step length
    w_step = min([abs(w - w_vtx) for w in w_hits])
    x_step = min([abs(x - x_vtx) for x in x_hits])
    step_length = np.sqrt(w_step**2 + x_step**2)

    return step_length

def find_radial_density_increase(x, y, bins=50, center=None, start_radius=0, debug=False, safety_r=5): # NOT A FEATURE
    if center is None:
        center = (np.mean(x), np.mean(y))
    
    r = np.sqrt((x - center[0])**2 + (y - center[1])**2)
    r = r[r > start_radius]
    if len(r) == 0:
        return safety_r
    
    r_sorted = np.sort(r)
    
    bin_edges = np.linspace(start_radius, r_sorted[-1], bins)
    counts, _ = np.histogram(r_sorted, bins=bin_edges)
    
    areas = np.pi * (bin_edges[1:]**2 - bin_edges[:-1]**2)
    densities = counts / areas
    
    diffs = np.diff(densities)
    increase_idx = np.argmax(diffs > 0)
    
    if increase_idx == 0 and diffs[0] <= 0:
        return safety_r
    
    return bin_edges[increase_idx]
def dE_dx(events, event_idx, min_hits=15, cone_angle=(5/6)*np.pi, debug=False):
    w_hits = events.reco_hits_w[event_idx]

    if len(w_hits) <= min_hits:
        return None
    
    x_hits = events.reco_hits_x_w[event_idx]
    w_vtx = events.neutrino_vtx_w[event_idx]
    x_vtx = events.neutrino_vtx_x[event_idx]
    adcs = events.reco_adcs_w[event_idx]
    
    pdg = events.mc_pdg[event_idx]

    if np.sign(np.mean(w_hits) - w_vtx) == -1:
        w_hits = 2 * w_vtx - np.array(w_hits)

    theta_0 = np.arctan2(np.mean(x_hits) - x_vtx, np.mean(w_hits) - w_vtx)
    theta_u = theta_0 + (cone_angle/2)
    theta_l = theta_0 - (cone_angle/2)

    angles = np.arctan2(x_hits - x_vtx, w_hits - w_vtx)
    distances = np.sqrt((w_hits - w_vtx) ** 2 + (x_hits - x_vtx) ** 2)

    r_start = np.partition(distances, 4)[4] if len(distances) >= 5 else 5
    
    testing_distance = find_radial_density_increase(w_hits, x_hits, center = (w_vtx, x_vtx), start_radius = r_start)
    branch_distance = max(r_start + 5, testing_distance)
    
    mask = (angles >= theta_l) & (angles <= theta_u) & (distances < branch_distance)
    if not np.any(mask):
        return None

    points = np.column_stack((w_hits[mask], x_hits[mask]))
    di = np.sqrt(((points[:, None, :] - points[None, :, :]) ** 2).sum(axis=2))
    dx = np.maximum(np.max(di), 0.48)

    val = np.sum(adcs[mask]) / dx

    return val

# booster features
def hit_count(events, event_idx):
    w_hits = events.reco_hits_w[event_idx]
    return len(w_hits)

def adc_sum(events, event_idx):
    adcs = events.reco_adcs_w[event_idx]
    return np.sum(adcs)

def hull_density(events, event_idx):
    w_hits = events.reco_hits_w[event_idx] 

    if len(w_hits) <= 2: # since there is no hull in that case
        return None
        
    x_hits = events.reco_hits_x_w[event_idx]
    points = np.column_stack((w_hits, x_hits))
    
    try:
        hull = ConvexHull(points)
        return len(points) / hull.volume
    except Exception as e:
        # Return None if any error occurs during hull computation
        return None

def curvature(events, event_idx, min_hits=15):
    """
    Booster feature to quantify track curvature and multiple scattering.
    
    Parameters:
        events: an object containing event hit information
        event_idx: index of the event in question
        min_hits: minimum number of hits required to compute the feature
        
    Returns:
        A combined curvature score (scaled) that incorporates both the curvature 
        (sum of absolute angle differences per unit length) and the RMS of the angle differences.
        Returns None if there are too few hits or no variation.
    """
    # Extract hit coordinates
    w_hits = np.array(events.reco_hits_w[event_idx])
    x_hits = np.array(events.reco_hits_x_w[event_idx])
    
    # Validate hit data
    if len(w_hits) != len(x_hits) or len(w_hits) < min_hits:
        return None

    # Calculate differences between successive points
    dw = np.diff(w_hits)
    dx = np.diff(x_hits)
    
    # Calculate the angle of each segment (using arctan2 gives the angle in radians)
    segment_angles = np.arctan2(dx, dw)
    
    # Compute differences between consecutive segment angles
    angle_diffs = np.diff(segment_angles)
    # Wrap the differences to [-π, π] to correctly account for periodicity
    angle_diffs = np.angle(np.exp(1j * angle_diffs))
    
    # Compute the length of each segment and the total track length
    segment_lengths = np.sqrt(dw**2 + dx**2)
    total_length = np.sum(segment_lengths)
    
    # Measure curvature: sum of absolute angle differences per unit length
    curvature = np.sum(np.abs(angle_diffs)) / total_length
    
    # Measure multiple scattering: RMS of the angle differences
    rms_angle = np.sqrt(np.mean(angle_diffs**2))
    
    # Combine the two measures with weighted contributions (e.g., 70% curvature, 30% RMS)
    combined_score = (0.7 * curvature + 0.3 * rms_angle) * 100  # scaling for convenience
    
    return combined_score

def adc_per_hit(events, event_idx):
    return adc_sum(events, event_idx)/hit_count(events, event_idx)

def max_adc_norm(events, event_idx):
    """
    Compute the maximum ADC value in each bin, where the ADC array
    is split into N_bins = min(50, hit_count(events, event_idx)) contiguous bins.
    
    Parameters:
        events: an object containing event hit information
        event_idx: index of the event
       
    Returns:
        A numpy array of maximum ADC values for each bin,
        or None if there are no ADC hits.
    """
    # Get ADC values for the event (assuming these are in a 1D array)
    adcs = np.array(events.reco_adcs_w[event_idx])
    n_hits = len(adcs)
    
    # If no hits, return None
    if n_hits == 0:
        return None

    # Determine the number of bins: use the smaller of 50 or the total number of hits
    n_bins = min(50, n_hits)
    
    # Split the ADC array into n_bins roughly equal parts
    adc_bins = np.array_split(adcs, n_bins)
    
    # For each bin, compute the maximum ADC value
    max_values = [np.max(bin_adc) for bin_adc in adc_bins if len(bin_adc) > 0]
    
    return np.argmax(max_values)/len(max_values)

def scatter_momentum(events, event_idx, min_hits=15):
    """
    Estimate the track momentum using multiple scattering analysis.
    
    This function calculates the RMS of the angle differences between consecutive
    track segments, then uses the Highland formula to estimate the momentum.
    Note: This method assumes the particle is relativistic (β ~ 1) and the track 
    is roughly continuous.
    
    Parameters:
        events: an object containing event hit information.
        event_idx: index of the event.
        min_hits: minimum number of hits required for the calculation.
        
    Returns:
        Estimated momentum (in MeV/c) of the track, or None if insufficient hits.
    """
    # Extract hit coordinates (assumes they are stored in 'reco_hits_w' and 'reco_hits_x_w')
    w_hits = np.array(events.reco_hits_w[event_idx])
    x_hits = np.array(events.reco_hits_x_w[event_idx])
    
    # Validate hit data
    if len(w_hits) != len(x_hits) or len(w_hits) < min_hits:
        return None

    # Calculate differences between successive hits (track segments)
    dw = np.diff(w_hits)
    dx = np.diff(x_hits)
    
    # Compute the angle of each segment (in radians)
    segment_angles = np.arctan2(dx, dw)
    
    # Calculate differences between consecutive segment angles
    angle_diffs = np.diff(segment_angles)
    # Wrap differences to [-pi, pi] to avoid discontinuities
    angle_diffs = np.angle(np.exp(1j * angle_diffs))
    
    # Compute the RMS of the angle differences
    rms_angle = np.sqrt(np.mean(angle_diffs**2))
    
    # Compute total track length (sum of segment lengths)
    segment_lengths = np.sqrt(dw**2 + dx**2)
    L = np.sum(segment_lengths)
    
    # Radiation length in liquid argon (approx. 14 cm)
    X0 = 14.0  # cm
    
    # Check to avoid division by zero
    if rms_angle == 0 or L <= 0:
        return None
    
    # Highland formula correction factor
    correction = 1 + 0.038 * np.log(L / X0)
    
    # Solve for momentum: p ≈ (13.6 MeV) * sqrt(L/X0) / (rms_angle * correction)
    momentum = (13.6 * np.sqrt(L / X0)) / (rms_angle * correction)
    
    return momentum

In [ ]:
"""
Getting sample data for training and testing phase
Also getting truth labels for each event_idx
"""
from tqdm import tqdm
def get_x_data(events, feature_func_arr):
    n = len(events.event_number)
    F_list = []
    print("Initialising...")

    for i in tqdm(range(n), desc="Processing events"):
        w_hits = events.reco_hits_w[i]
        if len(w_hits) <= 15:
            F_list.append([None] * len(feature_func_arr))
            continue
        x_hits = events.reco_hits_x_w[i]
        w_vtx = events.neutrino_vtx_w[i]
        x_vtx = events.neutrino_vtx_x[i]
        adcs = events.reco_adcs_w[i]
    
        if np.sign(np.mean(w_hits) - w_vtx) == -1:
            w_hits = 2 * w_vtx - np.array(w_hits)
    
        theta_0 = np.arctan2(np.mean(x_hits) - x_vtx, np.mean(w_hits) - w_vtx)
        theta_u = theta_0 + (np.pi*(5/6)/2)
        theta_l = theta_0 - (np.pi*(5/6)/2)
    
        angles = np.arctan2(x_hits - x_vtx, w_hits - w_vtx)
        distances = np.sqrt((w_hits - w_vtx) ** 2 + (x_hits - x_vtx) ** 2)
    
        r_start = np.partition(distances, 4)[4] if len(distances) >= 5 else 5
        
        testing_distance = find_radial_density_increase(w_hits, x_hits, center = (w_vtx, x_vtx), start_radius = r_start)
        branch_distance = max(r_start + 5, testing_distance)
        
        mask = (angles >= theta_l) & (angles <= theta_u) & (distances < branch_distance)
        if np.any(mask):
            F_vector = [feature(events, i) for feature in feature_func_arr]
            F_list.append(F_vector)
        else: F_list.append([None] * len(feature_func_arr))

    print('Complete!')
    return np.array(F_list)

def label_event_particle(events):
    pdgs = events.mc_pdg
    label_array = []

    for i, pdg in enumerate(pdgs):
        if pdg in [-13, 13]:
            label_array.append([i, 3]) # muon
        elif pdg in [-11, 11]:
            label_array.append([i, 2]) # electron
        elif pdg == 22:
            label_array.append([i, 1]) # photon
        else: label_array.append([i, 0]) # other

    return np.array(label_array)

# Here is the array of feature functions
features = [
    correlation,
    noise,
    rms,
    angle,
    line,
    q4,
    step_length,
    dE_dx,
    hit_count,
    adc_sum,
    hull_density,
    curvature,
    adc_per_hit,
    max_adc_norm,
    scatter_momentum
]

In [ ]:
"""
Actually training the BDT
    1. get x_train, y_train, x_test, y_test
    2. hyperparametrise the bdt
    3. save the bdt
"""
current_events = Events("CheatedRecoFile_0_new.root")

In [ ]:
x_train = get_x_data(current_events, features)
y_train = label_event_particle(current_events)

# Create a mask where rows are valid (i.e., they don't contain None)
mask = ~np.any(np.equal(x_train, None), axis=1)

# Apply the mask to filter both x and y
x_train = x_train[mask]
y_train = y_train[mask]

In [ ]:
print(x_train.shape)
print(y_train.shape)

In [ ]:
np.savez("15f_training.npz", x_train=x_train, y_train=y_train)

In [ ]:
current_events = Events("CheatedRecoFile_1_new.root") # testing

In [ ]:
x_test = get_x_data(current_events, features)
y_test = label_event_particle(current_events)

# Create a mask where rows are valid (i.e., they don't contain None)
mask = ~np.any(np.equal(x_test, None), axis=1)

# Apply the mask to filter both x and y
x_test = x_test[mask]
y_test = y_test[mask]

np.savez("15f_testing.npz", x_test=x_test, y_test=y_test)

In [ ]:
print(x_test.shape)
print(y_test.shape)

In [2]:
training = np.load("15f_training.npz", allow_pickle=True)
x_train = training["x_train"]
y_train = training["y_train"][:, 1]

param_grid = {
    'n_estimators': [50, 100, 200, 500],  # Number of trees
    'max_depth': [3, 5, 7, 10],  # Tree depth
    'learning_rate': [0.001, 0.01, 0.1, 0.3],  # Step size shrinkage
    'subsample': [0.6, 0.8, 1.0],  # Row sampling per tree
    'colsample_bytree': [0.6, 0.8, 1.0],  # Feature sampling per tree
    'gamma': [0, 0.1, 0.2, 0.5],  # Minimum loss reduction for split
    'reg_lambda': [0, 1, 10],  # L2 regularization
    'reg_alpha': [0, 1, 10]  # L1 regularization
}

In [4]:
bdt = xgb.XGBClassifier(eval_metric="logloss")

random_search = RandomizedSearchCV(
    estimator=bdt,
    param_distributions=param_grid,
    n_iter=50,  # Number of random samples to try
    scoring='accuracy',  # Optimize for classification accuracy
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1,  # Use all available CPU cores
    random_state=42
)

random_search.fit(x_train, y_train)

best_xgb = random_search.best_estimator_
print("Best hyperparameters found: ", random_search.best_params_)

joblib.dump(best_xgb, "optimized_15f.pkl")
print("Model saved successfully!")

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV] END colsample_bytree=1.0, gamma=0.1, learning_rate=0.001, max_depth=7, n_estimators=100, reg_alpha=0, reg_lambda=0, subsample=0.6; total time=   3.6s
[CV] END colsample_bytree=1.0, gamma=0.1, learning_rate=0.001, max_depth=7, n_estimators=100, reg_alpha=0, reg_lambda=0, subsample=0.6; total time=   3.6s
[CV] END colsample_bytree=1.0, gamma=0.1, learning_rate=0.001, max_depth=7, n_estimators=100, reg_alpha=0, reg_lambda=0, subsample=0.6; total time=   3.7s
[CV] END colsample_bytree=1.0, gamma=0.1, learning_rate=0.001, max_depth=7, n_estimators=100, reg_alpha=0, reg_lambda=0, subsample=0.6; total time=   3.6s
[CV] END colsample_bytree=1.0, gamma=0.1, learning_rate=0.001, max_depth=7, n_estimators=100, reg_alpha=0, reg_lambda=0, subsample=0.6; total time=   3.6s
[CV] END colsample_bytree=0.6, gamma=0, learning_rate=0.01, max_depth=10, n_estimators=500, reg_alpha=10, reg_lambda=1, subsample=1.0; total time=  17.7s
[CV] END 